# I. Input

## Raw data Input

### Full size data

In [23]:
import networkx as nx
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Directed graph G(V,E)
data = pd.read_csv("soc-sign-epinions.csv")

rng = np.random.default_rng(seed = 1234)
data['Weighted'] = rng.random(size = len(data))


# damping effect (paper)
d= 0.85

# total_time
total_time = 15

print(data)

N_max = max(data['FromNodeId'])

k = 1

n_iter = 10000

FileNotFoundError: [Errno 2] No such file or directory: 'soc-sign-epinions.csv'

In [ ]:
def full_size_data(data = data, n_iter=100, total_time = total_time):

  # Number of Nodes
  N = max(data['FromNodeId'])+1


  # the set of seed nodes with attitude 1 at time t
  S = np.zeros([n_iter, N])


  # Initial set of belief (random)
  X0 = rng.random(size=N)

  x = np.zeros([n_iter, N])
  x[0, :] = x[0, :] + X0

  # Initial set of altitude
  A = np.zeros([n_iter, N], dtype = np.int32)

  # Recommendation cycle time (random)
  T_l = rng.integers(low = 0, high = total_time, size = N)
  T_u = rng.integers(low = T_l, high = total_time, size = N)

  T = list(zip(T_l, T_u))

  df_graph = data

  return df_graph, N, x, A, T, X0, S


In [ ]:
df_graph, N, x, A, T, X0, S = full_size_data(data = data, n_iter=100, total_time = total_time)
# print(df_graph,"\n", N, "\n", x, "\n", A, "\n", T)

### Subset nodes data

In [ ]:
def subset_data(size ,n_iter=100, total_time = total_time):

  # Create the random graph
  G = nx.gnp_random_graph(size, 0.03, seed=0, directed = True)


  # Create a DataFrame type for graph
  data = pd.DataFrame(G.edges(), columns=['FromNodeId', 'ToNodeId'])
  data['Sign'] = rng.choice([-1, 1], size = len(data))
  data['Weighted'] = rng.random(size = len(data))


  # Number of Nodes
  N = max(data['FromNodeId'])+1


  # the set of seed nodes with attitude 1 at time t
  S = np.zeros([n_iter, N])


  # Initial set of belief (random)
  X0 = rng.random(size=N)

  x = np.zeros([n_iter, N])
  x[0, :] = x[0, :] + X0

  # Initial set of altitude
  A = np.zeros([n_iter, N], dtype = np.int32)

  # Recommendation cycle time (random)
  T_l = rng.integers(low = 1, high = total_time, size = N)
  T_u = rng.integers(low = T_l, high = total_time, size = N)

  T = list(zip(T_l, T_u))

  df_graph = data

  return df_graph, N, x, A, T, X0, S

In [ ]:
df_graph, N, x, A, T, X0, S = subset_data(3000, n_iter=100, total_time = total_time)

# print(df_graph,"\n", N, "\n", x, "\n", A, "\n", T)

## Matrix from raw data

In [ ]:
import networkx as nx
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from sklearn.preprocessing import normalize

def matrix_create(df_graph):
  G = nx.from_pandas_edgelist(df_graph, 'FromNodeId', 'ToNodeId', edge_attr='Sign', create_using=nx.DiGraph())
  adj_matrix = nx.adjacency_matrix(G).todense()

  # Number of individual
  N = max(df_graph['FromNodeId'])+1
  V = np.ones(len(G.nodes))

  # the set of seed nodes with attitude 1 at time t
  S = np.zeros([n_iter, N])

  # Create label matrix
  label_matrix = np.zeros(adj_matrix.shape, dtype=np.int32)
  for i, j in zip(df_graph['FromNodeId'], df_graph['ToNodeId']):
      label_matrix[i, j] = df_graph.loc[(df_graph['FromNodeId'] == i) & (df_graph['ToNodeId'] == j), 'Sign'].values

  # Create weighted matrix​
  weighted_matrix = np.zeros(adj_matrix.shape)
  for i, j in zip(df_graph['FromNodeId'], df_graph['ToNodeId']):
      weighted_matrix[i, j] = df_graph.loc[(df_graph['FromNodeId'] == i) & (df_graph['ToNodeId'] == j), 'Weighted'].values

  # the set of out-degree neighbours of vi
  H_out = {u : {v for v in G.successors(u)} for u in G.nodes}

  # the set of in-degree neighbours of vi
  H_in =  {v : {u for u in G.predecessors(v)} for v in G.nodes}

  # the set of in-degree foes of vi
  H_in_plus =  {v : {u for u in G.predecessors(v) if G[u][v]['Sign'] == 1} for v in G.nodes}

  # the set of in-degree foes of vi
  H_in_minus =  {v : {u for u in G.predecessors(v) if G[u][v]['Sign'] == -1} for v in G.nodes}

  return G, adj_matrix, N, V, label_matrix, weighted_matrix, H_out, H_in, H_in_plus, H_in_minus

# G, adj_matrix, N, V, label_matrix, weighted_matrix, H_out, H_in, H_in_plus, H_in_minus = matrix_create(df_graph)

# print(H_out)
# print(H_in)
# print(H_in_plus)
# print(H_in_minus)

# Process

## BUpdate: Belief Update Algorithm

In [ ]:
def BUpdate (G , X, A, Fa, t= 0):
  # Step 1: calculate alpha and peta
  alpha = []
  beta = []
  for i in range(N):
    alpha.append(len(H_in_plus[i])/len(H_in[i]))
    beta.append(len(H_in_minus[i])/len(H_in[i]))
  # print(alpha)
  # print(beta)

# Calculate positive update
  update_plus = 0
  update_minus = 0
  Pos = np.zeros(N)
  Neg = np.zeros(N)

  for i in range(N):
    if i in Fa:
      for j in range(N):
        if (j in H_in_plus[i]) and S[t][j] == 1:
          update_plus += (weighted_matrix[j][i] * (X[j] - X[i]))

        if j in H_in_minus[i] and S[t][j]== 1:
          update_minus += (weighted_matrix[j][i] * (X[j] - X[i]))


      Pos[i] = update_plus*alpha[i]
      Neg[i] = update_minus*beta[i]
      update_plus = 0
      update_minus = 0

  # Step2: Beief update
  X_update = np.zeros(N)
  update = X + Pos - Neg
  X_update[update<0] = 0
  X_update[update>0] = 1
  X_update[(update>0) & (update<1)] = update[(update>0) & (update<1)]
  # Step 3: Altitude update
  # A[t+1][:] = [np.random.binomial(1, x[1,j]) for j in range(len(G.nodes))]

  A_update = np.zeros(N, dtype = np.int32)
  for i in range(N):
    if (A[i] != 1):
      A_update[i] = np.random.binomial(1, X[i])
    else:
      A_update[i] = 1

  return X_update, A_update

### BUpdate testing

In [ ]:
Fa = {128, 129, 35, 227, 5, 133, 199, 104, 136, 259, 171, 12, 262, 264, 296, 89, 252, 93}
# X, A = BUpdate (G , x , A , Fa, t= 1)
# print(X, A)

## Sign PageRank

In [ ]:
def SignPR(G, X0, L, W, k=1, d=0.85):


  #iter = 0
  iter = 0

  SPR = np.zeros([n_iter, N])
  Sort = np.zeros([n_iter, N], dtype = np.int32)

  # 2: Calculate the the normalized matrix W~ = of W to make sum (i in Nodes) w~ = 1, forall v[j] in Node
  normalise_weighted_matrix = normalize(W,  norm="l1")


  # 3: Calculate Y based on Y = d.(W~*L);
  Y  = d*(normalise_weighted_matrix*L)

  # 4: for all vi in V do:
  # 5: SPR[iter][i] = x[0][i] .
  # 6: end for
  SPR[iter][:] += X0

  # 7: Sort[iter] = [1..n]
  Sort[iter][:] += np.arange(0,N)


  # 8: Sort[iter+1] = sort SPR[t][i] in a descending order.
  Sort[iter+1][:] = np.argsort(SPR[iter][:])[::-1]
  SPR_update = 0
  iter =-1


  # 9: while Sort[i][t+1] != Sort[i][t] do
  while (not (np.array_equal(Sort[iter][:], Sort[iter+1][:]))):
    iter +=1
    if iter == 100:
      break

    # 10: Sort[t] = Sort[t+1]
    if iter ==0:
      Sort[iter][:] = Sort[iter+1][:]

    # SPR_update = 0
    # 11: for all vi in V do
    # 12: SPR[i][iter+1] =   sum (j in H^out_i) (SPR[iter][i] - SPR[iter][j]) * y[i][j] + (1-d)/N
    # 13: end for
    for i in range(len(G.nodes)):
      for j in H_out[i]:
        SPR_update += (SPR[iter][i] - SPR[iter][j]) * Y[i][j]

      SPR[iter+1][i] = SPR_update + (1-d)/N
      # SPR_update = 0


    # 14: Sort[iter+1][:] = sort SPR[iter+1][:] in a descending order.
    Sort[iter+1][:] = np.argsort(SPR[iter+1][:])[::-1]

  # 17: S = the first k individuals in Sortt as seed nodes.
  Seed = np.zeros(len(G.nodes))
  for i in range(len(G.nodes)):
    if i in list(Sort[iter][:k]):
      Seed[i] += 1
  return Seed

In [ ]:
# Seed = []
# Seed = SignPR(G, X0, label_matrix, weighted_matrix, k=1, d=0.85)
# print(Seed)

### SPR test

In [ ]:
# # 1: t = 0.
# iter = 0
# SPR = np.zeros([10,5])
# Sort = np.zeros([10,5])

# # 2: Calculate the the normalized matrix W~ = of W to make sum (i in Nodes) w~ = 1, forall v[j] in Node
# normalise_weighted_matrix = normalize(weighted_matrix,  norm="l1")
# print("normalise_weighted_matrix \n" ,normalise_weighted_matrix)

# # 3: Calculate Y based on Y = d.(W~*L);
# Y  = d*(normalise_weighted_matrix*label_matrix)
# print("Y \n", Y)

# # 4: for all vi in V do:
# # 5: SPR[iter][i] = x[0][i] .
# # 6: end for
# SPR[iter][:] += X0
# print("SPR \n", SPR)

# # 7: Sort[iter] = [1..n]
# Sort[iter][:] += np.arange(0,len(G.nodes))
# print("Sort iter \n", Sort)

# # 8: Sort[iter+1] = sort SPR[t][i] in a descending order.
# Sort[iter+1][:] = np.argsort(SPR[iter][:])[::-1]
# print("Sort iter+1 \n", Sort)
# print("\n \n")
# SPR_update = 0
# iter = -1




# # 9: while Sort[i][t+1] != Sort[i][t] do
# while (not (np.array_equal(Sort[iter][:], Sort[iter+1][:]))):
#   iter +=1
#   if iter == 8:
#     break

#   # 10: Sort[t] = Sort[t+1]
#   print("Sort[iter][:] =", Sort[iter][:])
#   if iter ==0:
#     Sort[iter][:] = Sort[iter+1][:]

#   print("iter \n", iter)
#   print("Sort[iter][:] = Sort[iter+1][:] \n",Sort)
#   # SPR_update = 0

#   # 11: for all vi in V do
#   # 12: SPR[i][iter+1] =   sum (j in H^out_i) (SPR[iter][i] - SPR[iter][j]) * y[i][j] + (1-d)/N
#   # 13: end for
#   for i in range(len(G.nodes)):
#     for j in H_out[i]:
#       print("j = ", j)
#       SPR_update += (SPR[iter][i] - SPR[iter][j]) * Y[i][j]

#       print("SPR[iter][i] = ",SPR[iter][i])
#       print("SPR[iter][j] = ", SPR[iter][j])
#       print("Y[i][j] =", Y[i][j] )
#       print("SPR_update += (SPR[iter][i] - SPR[iter][j]) * Y[i][j] \n", SPR_update)

#     SPR[iter+1][i] = SPR_update + (1-d)/N
#     # SPR_update = 0

#     print("SPR[iter+1] = SPR_update + (1-d)/N \n", SPR)
#     print("\n i =", i+1)

#   # 14: Sort[iter+1][:] = sort SPR[iter+1][:] in a descending order.
#   Sort[iter+1][:] = np.argsort(SPR[iter+1][:])[::-1]

#   print("Sort[iter+1][:] = np.argsort(SPR[iter+1][:])[::-1] \n",np.argsort(SPR[iter+1][:])[::-1])
#   print("Sort iter+1 lan 2\n",Sort )

#   # 15: iter = iter + 1.

#   print("stop condition Sort[iter][:] \n", Sort[iter][:])
#   print("stop condition Sort[iter+1][:] \n",  Sort[iter+1][:])
#   print("stop condition Sort[iter][:] - Sort[iter+1][:] \n", Sort[iter][:] - Sort[iter+1][:])
#   print("\n \n")
#   # 16: end while

# # 17: S = the first k individuals in Sortt as seed nodes.
# Seed = Sort[iter][:k]
# print("S \n", Seed)

## InPro: Information Propagation Algorithm

In [47]:
### InPro
def InPro(G, x, A, label_matrix, T, t= 0):

  print("time = ",t)
  # 1: if t == 0 then
  if t == 0:
    # 2: S[t] = SPR(G; X0; L; k).

    #1 SignPR
    S[t][:] = SignPR(G, X0, label_matrix, weighted_matrix, k)

    #2 P_plus
    # S[t][:] = P_plus(label_matrix=label_matrix, weighted_matrix=weighted_matrix, k=k)
    
    #3 P_unsign
    # S[t][:] = P_unsign(label_matrix=label_matrix, weighted_matrix=weighted_matrix, k=k)

    #4 SRWR method
    # S[t][:] = SRWR_method(df_graph)
    
    A[t][:] = S[t][:]
  else:
    # 4: S = {vi | Ai,t = 1}.
    S[t][:] = A[t-1][:]
    # print("S[t][:] \n after update", S[t][:])
    if S[t][:].sum() == 0:
        return 0 
    print("Number of infected nodes: ", S[t][:].sum())
    infected_num.append(S[t][:].sum())

  # 6: The set of susceptible individuals who will receive ad recommendation is Fa = {vj | vj in V-S, t in [T^l_j; T^u_j]}.
  Fa = set([j for j in range(len(G.nodes)) if (V[j] - S[t][j] == 1) and (t >= T[j][0]) and (t <=T[j][1])])

  # 7: The set of susceptible individuals who will not receive ad recommendation is Fb = {vj | vj in V-S, t < T^l_j}.
  Fb = set([j for j in range(len(G.nodes)) if (V[j] - S[t][j] == 1) and (t < T[j][0]) ])

  # 8: if Fa is empty and Fb is non-empty then
  # 9: t = t + 1.
  # 10: InPro(G, Xt-1; At-1; L; T; t.
  if len(Fa) == 0 and len(Fb) > 0:
    # print("S[t][:] \n before update \n", S[t][:])
    # print("Belief initial", x[t][:])
    # print("Altitude initial", A[t][:])
    t += 1
    InPro(G, x, A, label_matrix, T, t)


  # 11: else if Fa is non-empty then
    # 12: t = t + 1.
    # 13: (Xt, At) = BUpdate(G; Xt-1; At-1; L).
  elif len(Fa) > 0:
    x[t][:], A[t][:] = BUpdate(G, x[t-1], A[t-1], Fa, t)
    # print("Belief Update", x[t][:])
    # print("Altitude Update", A[t][:])


    # 14: forall vi in (V - S) and Ait becomes 1 do
      # 15: Recover Ai;t from 1 to 0 with a probability of pi;t.

    non_seed = [index for index, value in enumerate(V-S[t]) if value > 0]
    for i in non_seed:
      if A[t][i] == 1:
        A[t][i] = np.random.binomial(1, x[t][i])
      # print("Unrecover proba", x[t][:])



    # 17: InProðG; Xt; At; L; T; tÞ.
    # print("S[t][:] \n", S[t][:])
    # print("Altitude Recover Update", A[t][:])
    t += 1
    if t == 1:
      return Fa
    InPro(G, x, A, label_matrix, T, t)


  # 18: else
  # 19: The information propagation terminates.
  else:
    return 0

In [ ]:
def P_plus(label_matrix, weighted_matrix, k):

    p_plus_matrix = np.array([weighted_matrix[i, j] if label_matrix[i, j] > 0 else 0 
                              for i in range(weighted_matrix.shape[0]) 
                              for j in range(weighted_matrix.shape[1])])
    
    # ind = np.argsort(p_plus_matrix)
    p_plus_matrix = p_plus_matrix.reshape(weighted_matrix.shape)
    p_plus_matrix = p_plus_matrix.sum(axis=1)
    top_k = np.argsort(p_plus_matrix)
    Seed = np.array([1 if i in top_k[-k:] else 0 for i in range(N)])
    
    return Seed

In [ ]:
def P_unsign(label_matrix, weighted_matrix, k):

    p_plus_matrix = weighted_matrix
    
    # ind = np.argsort(p_plus_matrix)
    p_plus_matrix = p_plus_matrix.reshape(weighted_matrix.shape)
    p_plus_matrix = p_plus_matrix.sum(axis=1)
    top_k = np.argsort(p_plus_matrix)
    Seed = np.array([1 if i in top_k[-k:] else 0 for i in range(N)])
    
    return Seed

In [ ]:
cd C:\Users\Admin\Documents

C:\Users\Admin\Documents


In [ ]:
# !pip install -r requirements.txt
# !git clone https://github.com/jinhongjung/srwr.git

In [ ]:
cd srwr

C:\Users\Admin\Documents\srwr


In [ ]:
from srwr.srwr import SRWR

def SRWR_method(df_graph):
    
    # Create an instance of SRWR
    srwr = SRWR()

    # Read the graph from the input path and perform normalization
    srwr.read_graph(df_graph)
    
    rd, rp, rn, residuals = srwr.query(seed=123, c=0.15, epsilon=1e-9, beta=0.5, gamma=0.5,
                                        max_iters=300, handles_deadend=True) # compute an SRWR query w.r.t. seed
    rd = rd.flatten()
    top_k = np.argsort(rd)
    Seed = np.array([1 if i in top_k[-k:] else 0 for i in range(N)])
    
    return Seed

In [32]:
weighted_matrix.shape
for k_index, k in enumerate([5, 10, 20]):
    print(k)

5
10
20


In [50]:
# df_graph, N, x, A, T, X0, S = subset_data(3000, n_iter=100, total_time = total_time)

# damping effect (paper)
d= 0.85

# total_time
total_time = 20

# 500 data points
df_graph, N, x, A, T, X0, S = subset_data(500, n_iter=100, total_time = total_time)
G, adj_matrix, N, V, label_matrix, weighted_matrix, H_out, H_in, H_in_plus, H_in_minus = matrix_create(df_graph)

# k comparison
infected_num_500_k = [[], [], []]

for k_index, k in enumerate([5, 10, 20]):
    infected_num = []
    InPro(G, x, A, label_matrix, T, t= 0)
    infected_num_500_k[k_index].append(infected_num)


# d comparison
infected_num_500_d = [[], [], []]
k = 5

for d_index, d in enumerate([0.15, 0.5, 0.85]):
    infected_num = []
    InPro(G, x, A, label_matrix, T, t= 0)
    infected_num_500_d[d_index].append(infected_num)



# 5000 data points
df_graph, N, x, A, T, X0, S = subset_data(5000, n_iter=100, total_time = total_time)
G, adj_matrix, N, V, label_matrix, weighted_matrix, H_out, H_in, H_in_plus, H_in_minus = matrix_create(df_graph)

# k comparison
d= 0.85

infected_num_5000_k = [[], [], []]

for k_index, k in enumerate([5, 10, 20]):
    infected_num = []
    InPro(G, x, A, label_matrix, T, t= 0)
    infected_num_5000_k[k_index].append(infected_num)


# d comparison
infected_num_5000_d = [[], [], []]

for d_index, d in enumerate([0.15, 0.50, 0.85]):
    infected_num = []
    InPro(G, x, A, label_matrix, T, t= 0)
    infected_num_5000_d[d_index].append(infected_num)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12228\773299618.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label_matrix[i, j] = df_graph.loc[(df_graph['FromNodeId'] == i) & (df_graph['ToNodeId'] == j), 'Sign'].values
C:\Users\Admin\AppData\Local\Temp\ipykernel_12228\773299618.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  weighted_matrix[i, j] = df_graph.loc[(df_graph['FromNodeId'] == i) & (df_graph['ToNodeId'] == j), 'Weighted'].values


time =  0
time =  1
Number of infected nodes:  5.0
time =  2
Number of infected nodes:  108.0
time =  3
Number of infected nodes:  144.0
time =  4
Number of infected nodes:  164.0
time =  5
Number of infected nodes:  184.0
time =  6
Number of infected nodes:  194.0
time =  7
Number of infected nodes:  206.0
time =  8
Number of infected nodes:  213.0
time =  9
Number of infected nodes:  217.0
time =  10
Number of infected nodes:  222.0
time =  11
Number of infected nodes:  225.0
time =  12
Number of infected nodes:  229.0
time =  13
Number of infected nodes:  236.0
time =  14
Number of infected nodes:  240.0
time =  15
Number of infected nodes:  241.0
time =  16
Number of infected nodes:  244.0
time =  17
Number of infected nodes:  246.0
time =  18
Number of infected nodes:  246.0
time =  19
Number of infected nodes:  252.0
time =  20
Number of infected nodes:  254.0
time =  0
time =  1
Number of infected nodes:  10.0
time =  2
Number of infected nodes:  107.0
time =  3
Number of infect

C:\Users\Admin\AppData\Local\Temp\ipykernel_12228\773299618.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label_matrix[i, j] = df_graph.loc[(df_graph['FromNodeId'] == i) & (df_graph['ToNodeId'] == j), 'Sign'].values
C:\Users\Admin\AppData\Local\Temp\ipykernel_12228\773299618.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  weighted_matrix[i, j] = df_graph.loc[(df_graph['FromNodeId'] == i) & (df_graph['ToNodeId'] == j), 'Weighted'].values


time =  0
time =  1
Number of infected nodes:  5.0
time =  2
Number of infected nodes:  168.0
time =  3
Number of infected nodes:  235.0
time =  4
Number of infected nodes:  279.0
time =  5
Number of infected nodes:  305.0
time =  6
Number of infected nodes:  328.0
time =  7
Number of infected nodes:  335.0
time =  8
Number of infected nodes:  348.0
time =  9
Number of infected nodes:  357.0
time =  10
Number of infected nodes:  366.0
time =  11
Number of infected nodes:  372.0
time =  12
Number of infected nodes:  377.0
time =  13
Number of infected nodes:  380.0
time =  14
Number of infected nodes:  384.0
time =  15
Number of infected nodes:  394.0
time =  16
Number of infected nodes:  398.0
time =  17
Number of infected nodes:  400.0
time =  18
Number of infected nodes:  402.0
time =  19
Number of infected nodes:  408.0
time =  20
Number of infected nodes:  410.0
time =  0
time =  1
Number of infected nodes:  10.0
time =  2
Number of infected nodes:  186.0
time =  3
Number of infect

In [64]:
pd.DataFrame(np.array(infected_num_500_k).reshape(3,20), index=[5,10,20])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
5,5.0,108.0,144.0,164.0,184.0,194.0,206.0,213.0,217.0,222.0,225.0,229.0,236.0,240.0,241.0,244.0,246.0,246.0,252.0,254.0
10,10.0,107.0,139.0,158.0,176.0,189.0,197.0,207.0,217.0,228.0,231.0,234.0,237.0,240.0,244.0,246.0,246.0,247.0,249.0,253.0
20,20.0,118.0,161.0,173.0,189.0,199.0,209.0,212.0,223.0,230.0,233.0,235.0,238.0,241.0,245.0,246.0,246.0,248.0,249.0,252.0


In [65]:
with pd.ExcelWriter("infected_number.xlsx") as writer:
  pd.DataFrame(np.array(infected_num_500_k).reshape(3,20), index=[5,10,20]).to_excel(writer, sheet_name='Result', startrow=0, startcol=0, header=True, index=True)
  pd.DataFrame(np.array(infected_num_500_d).reshape(3,20), index=[0.15,10,20]).to_excel(writer, sheet_name='Result', startrow=4, startcol=0, header=True, index=True)

  pd.DataFrame(np.array(infected_num_500_k).reshape(3,20), index=[5,10,20]).to_excel(writer, sheet_name='Result', startrow=8, startcol=0, header=True, index=True)
  pd.DataFrame(np.array(infected_num_500_d).reshape(3,20), index=[0.15,10,20]).to_excel(writer, sheet_name='Result', startrow=12, startcol=0, header=True, index=True)
